In [39]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.utils import plot_model

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
audio_path = "./audio_balanced"
json_path = "audio_data.json"

sr=44100

In [35]:
#Write MFCC data into JSON

def write_data(audio_path, json_path):
    #Dictionary of Genres and MFCC data
    mydict = {
        "genres":[],
        "mfcc":[]
    }
    
    #Generate MFCC for every song & writing in JSON
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(audio_path)):
        for file in filenames:
            #Generate MFCC
            song, sr = librosa.load(os.path.join(dirpath, file),duration=10)
            mfcc = librosa.feature.mfcc(y=song, sr=sr, n_mfcc=40)
            mfcc = mfcc.T
            
            #Data to be written in JSON
            mydict["genres"].append(i-1)
            mydict["mfcc"].append(mfcc.tolist())
            
        #Writing JSON 
        with open(json_path,'w') as f:
            json.dump(mydict, f)
        f.close()
            

In [36]:
#READ MFCC data from JSON

def read_data(json_path):
    #Open JSON
    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close
    
    #Load data in numpy arrays for compatability reasons
    X = np.array(data["mfcc"])
    y = np.array(data["genres"])

In [ ]:
#Use random split to seperate data into training, validation and test

def split_dataset(inputs, targets, split_size):
    #Train, Validation & Test set
    inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size = split_size)
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs_train, targets_train, test_size = split_size)

    inputs_train = inputs_train[...,np.newaxis]
    inputs_val = inputs_val[...,np.newaxis]
    inputs_test=  inputs_test[..., np.axis]
    
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [ ]:
#Use stratified split to seperate data into training, validation and test

def stratified_split_dataset(inputs, targets, split_size):
    #Train, Validation & Test set using stratified sampling
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split_size, random_state=42)
    
    train_val_indices, test_indices = next(sss.split(inputs, targets))
    inputs_train_val, targets_train_val = inputs[train_val_indices], targets[train_val_indices]
    
    sss_train = StratifiedShuffleSplit(n_splits=1, test_size=split_size, random_state=43)
    train_indices, val_indices = next(sss_train.split(inputs_train_val, targets_train_val))
    
    inputs_train, targets_train = inputs_train_val[train_indices], targets_train_val[train_indices]
    inputs_val, targets_val = inputs_train_val[val_indices], targets_train_val[val_indices]
    inputs_test, targets_test = inputs[test_indices], targets[test_indices]
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [ ]:
#Design the model
def design_mdoel(input_shape):
    
    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(32, (2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'), 
        tf.keras.layers.Dense(len(np.unique(targets)), activation='softmax')
    ])

    return model

In [ ]:
#Used to plot performance history of model
def plot_performance(hist):
    
    acc = hist.history['acc']
    val_acc = hist.history['val_acc']
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()

    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
#=============MAIN ================
#Process data
write_data(audio_path, json_path)

inputs, targets = read_data(json_path)

Xtrain, Xval, Xtest, ytrain, yval, ytest = stratified_split_dataset(inputs, targets, 0.2)


In [ ]:
#Model Creation
input_shape = (Xtrain.shape[1], Xtrain.shape[2], 1)
model = design_model(input_shape)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.001),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['acc'])

In [ ]:
#Model Summary
model.summary()

In [ ]:
#Plot Model
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(Xtrain, ytain,
                   validation_data = (Xval, yval),
                   epochs = 100,
                   batch_size = 32)

In [ ]:
#Plot Performance
plot_performance(history)

In [ ]:
model.save('MFCC Num Model')

In [ ]:
model.save('MFCC Num Model.h5', save_format='h5')